In [1]:
import pandas as pd
import torch
import numpy as np
import os
import sys

DATA_PATH = '../../data/matches-processed/cs2/hetero-graph/'

### Concat single matches

In [20]:
file_names = [
    '100110.pt',
    '100111.pt',
    '100112.pt',
    '100113.pt'
]

lens = 0
concat_list = []

for file in file_names:
    data = torch.load(DATA_PATH + file, weights_only=False)
    lens += len(data)
    concat_list.extend(data)

In [21]:
print(lens)
print(len(concat_list))

32364
32364


In [22]:
torch.save(concat_list, DATA_PATH + '/concat/100110-100113.pt')

### Create bomb planted datasets

In [24]:
file_names = [
    '100000-100009.pt',
    '100010-100019.pt',
    '100020-100029.pt',
    '100030-100039.pt',
    '100040-100049.pt',
    '100050-100059.pt',
    '100060-100069.pt',
    '100070-100079.pt',
    '100080-100089.pt',
    '100090-100099.pt',
    '100100-100109.pt',
    '100110-100113.pt'
]

for file_name in file_names:

    match_graphs = torch.load(DATA_PATH + '/concat/' + file_name, weights_only=False)
    bomb_planted = []

    for graph in match_graphs:
        if graph.y['is_bomb_planted_at_A_site'] == np.float16(1) or graph.y['is_bomb_planted_at_B_site'] == np.float16(1):
            bomb_planted.append(graph)

    torch.save(bomb_planted, DATA_PATH + '/bomb_planted/' + file_name)
    print(f'Saved {file_name}')

Saved 100000-100009.pt
Saved 100010-100019.pt
Saved 100020-100029.pt
Saved 100030-100039.pt
Saved 100040-100049.pt
Saved 100050-100059.pt
Saved 100060-100069.pt
Saved 100070-100079.pt
Saved 100080-100089.pt
Saved 100090-100099.pt
Saved 100100-100109.pt
Saved 100110-100113.pt


In [9]:
bomb_file_names = [
    '100000-100009.pt',
    '100010-100019.pt',
    '100020-100029.pt',
    '100030-100039.pt',
    '100040-100049.pt',
    '100050-100059.pt',
    '100060-100069.pt',
    '100070-100079.pt',
    '100080-100089.pt',
    '100090-100099.pt',
    '100100-100109.pt',
]

for file_name in bomb_file_names:

    match_graphs = torch.load(DATA_PATH + '/bomb_planted/' + file_name, weights_only=False)
    print(f'Graphs in {file_name}: {len(match_graphs)}')

Graphs in 100000-100009.pt: 18199
Graphs in 100010-100019.pt: 14658
Graphs in 100020-100029.pt: 17705
Graphs in 100030-100039.pt: 17674
Graphs in 100040-100049.pt: 19612
Graphs in 100050-100059.pt: 15886
Graphs in 100060-100069.pt: 16954
Graphs in 100070-100079.pt: 19228
Graphs in 100080-100089.pt: 19015
Graphs in 100090-100099.pt: 17290
Graphs in 100100-100109.pt: 15621


### Create training dataset

In [25]:
file_names = [
    '100000-100009.pt',
    '100010-100019.pt',
    '100020-100029.pt',
    '100030-100039.pt',
    '100040-100049.pt',
    '100050-100059.pt',
    '100060-100069.pt',
    '100070-100079.pt',
]

train_data = []

for file in file_names:

    match_graphs = torch.load(DATA_PATH + '/concat/' + file, weights_only=False)
    train_data.extend(match_graphs)
    print(f'Loaded {file}')

torch.save(train_data, DATA_PATH + '/train.pt')
    

Loaded 100000-100009.pt
Loaded 100010-100019.pt
Loaded 100020-100029.pt
Loaded 100030-100039.pt
Loaded 100040-100049.pt
Loaded 100050-100059.pt
Loaded 100060-100069.pt
Loaded 100070-100079.pt


In [26]:
ct_wins = 0
for grapf in train_data:
    if grapf.y['CT_wins'] == np.float16(1):
        ct_wins += 1

print(ct_wins / len(train_data))

0.460989681602123


In [27]:
planted = 0
for grapf in train_data:
    if grapf.y['is_bomb_planted_at_A_site'] == np.float16(1) or grapf.y['is_bomb_planted_at_B_site'] == np.float16(1):
        planted += 1

print(planted / len(train_data))

0.1904578614098638


### Create validation dataset

In [28]:
file_names = [
    '100080-100089.pt',
    '100090-100099.pt',
    '100110-100113.pt',
]

val_data = []

for file in file_names:

    match_graphs = torch.load(DATA_PATH + '/concat/' + file, weights_only=False)
    val_data.extend(match_graphs)
    print(f'Loaded {file}')

torch.save(val_data, DATA_PATH + '/val.pt')
    

Loaded 100080-100089.pt
Loaded 100090-100099.pt
Loaded 100110-100113.pt


In [29]:
ct_wins = 0
for graph in val_data:
    if graph.y['CT_wins'] == np.float16(1):
        ct_wins += 1

print(ct_wins / len(val_data))

0.4753377821166045


### Output variable validation

In [30]:
for file in os.listdir(DATA_PATH + '/bomb_planted/'):

    train_data = torch.load(DATA_PATH + f'/bomb_planted/{file}', weights_only=False)

    ct_wins = 0
    for graph in train_data:
        if graph.y['CT_wins'] == np.float16(1):
            ct_wins += 1

    print(file + ': ' + str(ct_wins / len(train_data)))

100000-100009.pt: 0.1513700155936734
100010-100019.pt: 0.13963410224889725
100020-100029.pt: 0.14991181657848324
100030-100039.pt: 0.15914067585962005
100040-100049.pt: 0.17005404135338345
100050-100059.pt: 0.20787236520321417
100060-100069.pt: 0.12294350842418236
100070-100079.pt: 0.12441417234268574
100080-100089.pt: 0.1735626928791862
100090-100099.pt: 0.18470373270686505
100100-100109.pt: 0.1546796560465232
100110-100113.pt: 0.20327820869310698
